In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt



In [9]:
%%bash

# Variables
REPO_URL="https://github.com/LIMAMMohamedlimam/sammed-lite.git"
CLONE_DIR="temp_repo"
TARGET_DIR="./"
git clone "$REPO_URL" "$CLONE_DIR"

# Create target directory if it doesn't exist
mkdir -p "$TARGET_DIR"

# Copy all contents (including hidden files)
cp -r "$CLONE_DIR"/. "$TARGET_DIR"/

# Delete cloned repo directory
rm -rf "$CLONE_DIR"

echo "Done: copied repo content into $TARGET_DIR"

Done: copied repo content into ./


Cloning into 'temp_repo'...


In [ ]:
!ls -R

.:
lite-sammed2d.py     sample_data	   segment_anything
SAMMed2D-lite.ipynb  sam_vit_b_01ec64.pth

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md

./segment_anything:
automatic_mask_generator.py  __init__.py  predictor.py
build_sam.py		     modeling	  utils

./segment_anything/modeling:
Adapted_Block.py  image_encoder.py  mask_decoder.py    sam.py
common.py	  __init__.py	    prompt_encoder.py  transformer.py

./segment_anything/utils:
amg.py	__init__.py  onnx.py  transforms.py


In [ ]:
from segment_anything.modeling import (
    Sam,
    ImageEncoderViT,
    MaskDecoder,
    PromptEncoder,
    TwoWayTransformer,
    adapter
)

ModuleNotFoundError: No module named 'Adapted_Block'

In [27]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Adapter layer implementation


### Custom Transformer Block  (Adapter_layer injected)

In [ ]:

custom_image_encoder = ImageEncoderViT(
    transformer_block=adapter
)

TypeError: ImageEncoderViT.__init__() got an unexpected keyword argument 'transformer_block'

## Medical Image Dataset

In [33]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


--2025-11-22 13:22:26--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.142.20, 108.157.142.40, 108.157.142.74, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.142.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  23.2MB/s    in 6.3s    

2025-11-22 13:22:32 (57.1 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [34]:
sam_checkpoint = "sam_vit_b_01ec64.pth"

In [35]:
# test
from segment_anything.build_sam import sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)



In [38]:
for name,parm in sam.image_encoder.named_parameters() : 
    # if "Adapter" in name : 
        print(f"name : {name}")


name : pos_embed
name : patch_embed.proj.weight
name : patch_embed.proj.bias
name : blocks.0.norm1.weight
name : blocks.0.norm1.bias
name : blocks.0.attn.rel_pos_h
name : blocks.0.attn.rel_pos_w
name : blocks.0.attn.qkv.weight
name : blocks.0.attn.qkv.bias
name : blocks.0.attn.proj.weight
name : blocks.0.attn.proj.bias
name : blocks.0.norm2.weight
name : blocks.0.norm2.bias
name : blocks.0.mlp.lin1.weight
name : blocks.0.mlp.lin1.bias
name : blocks.0.mlp.lin2.weight
name : blocks.0.mlp.lin2.bias
name : blocks.1.norm1.weight
name : blocks.1.norm1.bias
name : blocks.1.attn.rel_pos_h
name : blocks.1.attn.rel_pos_w
name : blocks.1.attn.qkv.weight
name : blocks.1.attn.qkv.bias
name : blocks.1.attn.proj.weight
name : blocks.1.attn.proj.bias
name : blocks.1.norm2.weight
name : blocks.1.norm2.bias
name : blocks.1.mlp.lin1.weight
name : blocks.1.mlp.lin1.bias
name : blocks.1.mlp.lin2.weight
name : blocks.1.mlp.lin2.bias
name : blocks.2.norm1.weight
name : blocks.2.norm1.bias
name : blocks.2.att

## Model implementation SAM-Med2D-Lite

In [ ]:
class SAMMed2DLite(nn.Module):
    def __init__(
        self,
        sam_model : Sam,
        embed_dim=768,
    ):
        
        """Lite version of SAM-Med2D with adapter layers for medical image segmentation."""
        super().__init__()
        self.sam = sam_model

        #Freeze image encoder params except the adapter layer
        for name , params in self.sam.image_encoder.parameters() :
            if adap
